# Imports, Constants and Setup

In [1]:
import os
import requests
import vec_geohash
from tqdm import tqdm
import geopandas as gpd
from osmtogeojson import osmtogeojson
import logging

# constants
ZOOM = 16
AREA_NAME = 'Italia'

# logging setup
logging.basicConfig(filename='osm_area_extraction_logger.txt',
                    filemode='a',
                    format='%(asctime)s | %(levelname)s: %(message)s',
                    datefmt='%m/%d/%Y%I:%M:%S %p',
                    level=logging.INFO)
logging.info("OSM Area Extraction")

# results folder setup
RES_FOLDER = f'osm_area_data_extracted'
if not os.path.exists(RES_FOLDER):
    os.mkdir(RES_FOLDER)

# Helper functions

In [2]:
def query_osm_for_solar_panels_by_bbox(lat_min, lon_min, lat_max, lon_max):
    """Queries OSM for solar generators in bounding box."""
    osm_query = f"""
                    [out:json][timeout:3000];
                    way{lat_min, lon_min, lat_max, lon_max}["generator:source"="solar"];
                    (._;>;);
                    out; 
                """
    response = requests.get("http://overpass-api.de/api/interpreter", params={'data': osm_query})
    return response


def query_osm_for_solar_panels_by_area(area_name='Italia'):
    """Queries OSM for solar generators in bounding box."""
    osm_query = f"""
                    [out:json][timeout:3000];
                    area["name"="{area_name}"];
                    way["generator:source"="solar"](area);
                    (._;>;);
                    out skel; 
                """
    response = requests.get("http://overpass-api.de/api/interpreter", params={'data': osm_query})
    return response


def parse_response(response):
    """Parses the response if status is good else raises an error."""
    logging.info(f'Parsing data')
    response.raise_for_status()
    data_json = response.json()
    osm_data = osmtogeojson.process_osm_json(data_json)
    return osm_data


def load_data_to_gdf(data_json):
    """Transforms dict data into GeoDataFrame. Filters to just polygons, calculates area and tile boundary columns."""
    gdf = gpd.GeoDataFrame.from_features(data_json, crs='epsg:4326', columns=['@id', 'geometry'])
    gdf = gdf[gdf.geometry.type == 'Polygon']
    gdf['area_m2'] = gdf.geometry.to_crs('epsg:3857').area.abs() * 1_000_000

    bounds = gdf.geometry.bounds.values
    gdf[['min_tile_x', 'min_tile_y', 'max_tile_x', 'max_tile_y']] = vec_geohash.lat_lon_bounds_to_tile_range(bounds, ZOOM)
    return gdf


def process_area_for_solar_powerplants(area_name):
    """Extracting solar powerplants data for given area, processing it and then saving it."""

    # querying OSM
    logging.info(f"Starting extraction for {area_name}")
    response = query_osm_for_solar_panels_by_area(area_name)

    # creating dataset
    logging.info(f"Extraction finished - processing data")
    data_json = parse_response(response)
    gdf = load_data_to_gdf(data_json)

    # saving data
    logging.info(f"Processing finished - saving data")
    path = f'{RES_FOLDER}/osm_data_{area_name}.geofeather'
    gdf.to_feather(path)

    logging.info(f"Saving data finished - area {area_name} done")
    return gdf

        

# Extraction of data in Area

In [3]:
gdf = process_area_for_solar_powerplants(area_name='Italia')
gdf

,@id,geometry,area_m2,min_tile_x,min_tile_y,max_tile_x,max_tile_y
0,way/36196018,"POLYGON ((13.44121 46.02533, 13.44142 46.02530...",7.556920e+08,35214,23308,35214,23308
1,way/36425969,"POLYGON ((13.48298 46.11711, 13.48324 46.11710...",4.546853e+09,35222,23284,35222,23284
2,way/36426151,"POLYGON ((13.41266 46.09006, 13.41243 46.09004...",9.289774e+09,35209,23291,35209,23291
3,way/36426406,"POLYGON ((13.43839 46.00887, 13.43740 46.00891...",1.248245e+10,35214,23312,35214,23312
4,way/36668332,"POLYGON ((13.43674 46.04603, 13.43669 46.04695...",3.297062e+09,35214,23302,35214,23303
...,...,...,...,...,...,...,...
31329,way/1199998712,"POLYGON ((12.47741 44.08776, 12.47740 44.08775...",3.324967e+07,35039,23807,35039,23807
31330,way/1199998713,"POLYGON ((12.47740 44.08774, 12.47740 44.08772...",3.312726e+07,35039,23807,35039,23807
31331,way/1199998714,"POLYGON ((12.47740 44.08772, 12.47740 44.08771...",3.339069e+07,35039,23807,35039,23807
31332,way/1199998715,"POLYGON ((12.47740 44.08770, 12.47739 44.08769...",3.324645e+07,35039,23807,35039,23807
